# Run simulations

In [13]:
# Add project root to Python path
import sys
from pathlib import Path

project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

# Import necessary libraryes
import os
import joblib
import numpy as np
import pandas as pd
import plotnine as p9
import psutil
from src.population_density_networks import model

## Modeling Parameters

In [6]:
# Fixed global parameters
N = 1000
K_CAP_MEAN = 50
K_CAP_SD = 0.5 * K_CAP_MEAN
RADIUS = 1.0

# Varying parameters
DENSITIES = [1e-4, 1e4]
# DENSITIES = np.logspace(-5, 5, 21, base=10)

# Simulation parameters
SIMULATION_ROUNDS = 2 * N * K_CAP_MEAN
SIMULATION_REPLICATES = 50
N_JOBS = psutil.cpu_count(logical=True) - 1

# Save paths
OUTPUT_DIR = '../data_derived'
SAVE_PATH = f'{OUTPUT_DIR}/select_density_results.csv'

## Run Simulations

In [7]:
#####################
# Run simulations
#####################
# Generate all parameter combinations
param_combinations = [
    (
        density,
        replicate, 
        N,
        K_CAP_MEAN,
        K_CAP_SD,
        RADIUS,
        SIMULATION_ROUNDS,
        True,  # Return network
    )
    for density in DENSITIES
    for replicate in range(SIMULATION_REPLICATES)
]

print(f"🚀 Running {len(param_combinations)} simulations in parallel...")

# Run simulations in parallel
parallel_jobs = min(N_JOBS, len(param_combinations))
results_list = joblib.Parallel(n_jobs=parallel_jobs, verbose=10)(
    joblib.delayed(model.run_single_simulation)(*params) for params in param_combinations
)

# Convert results to DataFrame
results = pd.DataFrame(results_list)

🚀 Running 100 simulations in parallel...


[Parallel(n_jobs=13)]: Using backend LokyBackend with 13 concurrent workers.
[Parallel(n_jobs=13)]: Done   6 tasks      | elapsed:   40.5s
[Parallel(n_jobs=13)]: Done  15 tasks      | elapsed:  1.3min
[Parallel(n_jobs=13)]: Done  24 tasks      | elapsed:  1.3min
[Parallel(n_jobs=13)]: Done  35 tasks      | elapsed:  1.9min
[Parallel(n_jobs=13)]: Done  46 tasks      | elapsed:  2.6min
[Parallel(n_jobs=13)]: Done  59 tasks      | elapsed:  3.0min
[Parallel(n_jobs=13)]: Done  72 tasks      | elapsed:  3.5min
[Parallel(n_jobs=13)]: Done  86 out of 100 | elapsed:  4.0min remaining:   38.6s
[Parallel(n_jobs=13)]: Done  97 out of 100 | elapsed:  4.3min remaining:    8.0s
[Parallel(n_jobs=13)]: Done 100 out of 100 | elapsed:  4.4min finished


In [10]:
results

,n,k_cap_mean,k_cap_sd,radius,simulation_rounds,population_density,replicate,network_density,network_is_connected,network_num_components,network_largest_component_size,network_avg_shortest_path,network_diameter,network_clustering_coef,network_modularity,network_assortativity,social_network
0,1000,50,25.0,1.0,100000,0.0001,0,0.043792,True,1,1000,2.279778,4,0.171749,0.373500,0.061143,0 1 2 3 4 5 6 7 8...
1,1000,50,25.0,1.0,100000,0.0001,1,0.042895,True,1,1000,2.298729,4,0.173112,0.360644,0.064336,0 1 2 3 4 5 6 7 8...
2,1000,50,25.0,1.0,100000,0.0001,2,0.042735,True,1,1000,2.287435,4,0.168809,0.342344,0.050811,0 1 2 3 4 5 6 7 8...
3,1000,50,25.0,1.0,100000,0.0001,3,0.042228,True,1,1000,2.304759,4,0.171078,0.362423,0.064324,0 1 2 3 4 5 6 7 8...
4,1000,50,25.0,1.0,100000,0.0001,4,0.042703,True,1,1000,2.295023,4,0.168958,0.384783,0.060114,0 1 2 3 4 5 6 7 8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1000,50,25.0,1.0,100000,10000.0000,45,0.046807,True,1,1000,2.115734,3,0.063197,0.139796,0.112488,0 1 2 3 4 5 6 7 8...
96,1000,50,25.0,1.0,100000,10000.0000,46,0.048094,True,1,1000,2.098038,3,0.062365,0.131885,0.104805,0 1 2 3 4 5 6 7 8...
97,1000,50,25.0,1.0,100000,10000.0000,47,0.047411,True,1,1000,2.104593,3,0.062006,0.137020,0.109864,0 1 2 3 4 5 6 7 8...
98,1000,50,25.0,1.0,100000,10000.0000,48,0.047532,True,1,1000,2.107037,3,0.062852,0.141250,0.116600,0 1 2 3 4 5 6 7 8...


## Save results

In [ ]:
#####################
# Save data
#####################
# If `social_network` columns is present, write each to file
if 'social_network' in results.columns:
    os.makedirs(f'{OUTPUT_DIR}/full_social_networks/', exist_ok=True)
    for i, row in results.iterrows():
        density = row['population_density']
        row['social_network'].to_csv(f'{OUTPUT_DIR}/full_social_networks/social_network-density_{density}-replicate_{i}.csv')
        
# Write to file
results.to_csv(SAVE_PATH, index=False)